# Prepare dataloader for 3D CNN (30 classes only)

In [4]:
#%%python3 -m pip install --upgrade pip


In [5]:
#%pip install tensorflow_docs --upgrade
!pip install mediapipe=='0.10.9'
!pip install opencv-python=='4.8.0.76'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.1 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from utils import VideoDataset, validate_model, train_model, display_frames, display_learning_dynamic, classification_model_metrics

In [9]:
import cv2
cv2.__version__

'4.8.0'

In [10]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [11]:
use_cuda = True
seed = 1

train_batch_size = 1
test_batch_size = 1
val_batch_size = 1
IMG_SIZE = 112

limit_list = []

min_frame_count = 25


annotations_file = "/content/drive/MyDrive/slovo/SLOVO_DF_SHORT.tsv" #"/home/jupyter/mnt/s3/rsl-videos/slovo/slovo_annotations/SLOVO_DATAFRAME.tsv" #
video_dir = "/content/drive/MyDrive/slovo/animals" # "/home/jupyter/mnt/s3/rsl-videos/slovo/slovo" #

face_detection_model = '/content/drive/MyDrive/slovo/face_detection_yunet_2023mar.onnx'


In [12]:
video_labels = pd.read_csv(annotations_file, sep='\t')
video_labels['group_rank'] = video_labels.groupby(['text']).cumcount()+1;
video_labels['dataset'] = np.where(video_labels['group_rank']<17,'train', np.where(video_labels['group_rank']<19,'val', 'test'))
if len(limit_list)>0:
  video_labels = video_labels[video_labels.text.isin(limit_list)]
video_labels.tail(5)

,Unnamed: 0,attachment_id,text,begin,end,group_rank,dataset
595,55,25b5fb58-46dd-4fd6-9928-734460795b22,мышь,22,83,20,test
596,56,336e0e7a-ff68-427c-9a77-52626b81edd1,паук,42,74,19,test
597,57,d0ec4c00-6ff5-4089-ac9d-5a33a76fb037,паук,3,70,20,test
598,58,59ae408b-f87a-422a-bc57-2ff5af28ee57,бабочка,2,74,19,test
599,59,e421d77c-1dcd-4124-b2a4-b2415569a4dd,бабочка,63,131,20,test


In [13]:
classes = tuple(video_labels.text.unique())
n_class = len(classes)
print(f"Total number of classes: {n_class}. Example classes: {classes[:5]}")

Total number of classes: 30. Example classes: ('пингвин', 'жираф', 'лягушка', 'бегемот', 'козел')


In [14]:
classes.index('бегемот')

3

In [15]:
train_kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
test_kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
val_kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

In [14]:
test_dataloader = DataLoader(VideoDataset(video_labels, video_dir, IMG_SIZE, min_frame_count, classes, face_detection_model, ds_type = "test"), batch_size=test_batch_size, shuffle=True, **test_kwargs)

In [15]:
torch.save(test_dataloader, '/content/drive/MyDrive/slovo/dataloaders/color_frames_test_dataloader.pth')

In [16]:
val_dataloader = DataLoader(VideoDataset(video_labels, video_dir, IMG_SIZE, min_frame_count, classes, face_detection_model, ds_type = "val"), batch_size=val_batch_size, shuffle=True, **val_kwargs)

In [17]:
torch.save(val_dataloader, '/content/drive/MyDrive/slovo/dataloaders/color_frames_val_dataloader.pth')

In [16]:
train_dataloader = DataLoader(VideoDataset(video_labels, video_dir, IMG_SIZE, min_frame_count, classes, face_detection_model, ds_type = "train"), batch_size=train_batch_size, shuffle=True, **train_kwargs)

In [17]:
torch.save(train_dataloader, '/content/drive/MyDrive/slovo/dataloaders/color_frames_train_dataloader.pth')